Load data from dataset

In [2]:
#read train.json data 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pandas.io.json import json_normalize
import warnings
warnings.filterwarnings('ignore')

#read train.json data
train_data = pd.read_json('train.json')

#read test.json data
test_data = pd.read_json('test.json')

#Pack the prediction result into a csv file
def pack_result(result):
    result = pd.DataFrame(result)
    result.columns = ['ID', 'Prediction']
    result.to_csv('result.csv', index=False)



In [3]:
train_data.head()
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25793 entries, 0 to 25792
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   authors   25793 non-null  object
 1   year      25793 non-null  int64 
 2   abstract  25793 non-null  object
 3   venue     25793 non-null  object
 4   title     25793 non-null  object
dtypes: int64(1), object(4)
memory usage: 1007.7+ KB


In [4]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   identifier  800 non-null    int64 
 1   coauthors   800 non-null    object
 2   year        800 non-null    int64 
 3   abstract    800 non-null    object
 4   venue       800 non-null    object
 5   title       800 non-null    object
dtypes: int64(2), object(4)
memory usage: 37.6+ KB
